In [ ]:
import pandas as pd
import regex as re

In [ ]:
link_rt = 'https://raw.githubusercontent.com/Sebastiao199/Project3MRS/main/OtherTables/out4.csv'
link_imdb = 'https://raw.githubusercontent.com/Sebastiao199/Project3MRS/main/OtherTables/tprn_10.csv'
imdb = pd.read_csv(link_imdb)
rt = pd.read_csv(link_rt)

In [ ]:
imdb

,Unnamed: 0,tconst,Movie title,Year,Genres,nconst,category,IMDb rating,Nb votes,Staff name
0,4316,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0001273,actress,6.2,24922.0,Lillian Gish
1,4317,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0550615,actress,6.2,24922.0,Mae Marsh
2,4318,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0910400,actor,6.2,24922.0,Henry B. Walthall
3,4319,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0178270,actress,6.2,24922.0,Miriam Cooper
4,4320,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0000428,director,6.2,24922.0,D.W. Griffith
...,...,...,...,...,...,...,...,...,...,...
68493,3364581,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",nm2933542,actor,6.2,11042.0,Boyd Holbrook
68494,3364582,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",nm0717709,actress,6.2,11042.0,Kelly Reilly
68495,3364583,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",nm0677944,actor,6.2,11042.0,Alistair Petrie
68496,3364584,tt9907782,The Cursed,2021,"Fantasy,Horror,Mystery",nm3646923,actress,6.2,11042.0,Roxane Duran


In [ ]:
## Get one row per movie. As we are doing the join with RT based on director and movie title, we kept only the row of the director
cond1= imdb['category']=='director'
imdb1 = imdb[cond1]
imdb1


,Unnamed: 0,tconst,Movie title,Year,Genres,nconst,category,IMDb rating,Nb votes,Staff name
4,4320,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0000428,director,6.2,24922.0,D.W. Griffith
9,10770,tt0006864,Intolerance,1916,"Drama,History",nm0000428,director,7.7,15692.0,D.W. Griffith
15,25195,tt0009968,Broken Blossoms,1919,"Drama,Romance",nm0000428,director,7.3,10437.0,D.W. Griffith
22,26838,tt0010323,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,director,8.0,64319.0,Robert Wiene
27,31277,tt0011237,The Golem,1920,"Fantasy,Horror",nm0091380,director,7.2,7825.0,Carl Boese
...,...,...,...,...,...,...,...,...,...,...
68477,3363277,tt9893250,I Care a Lot,2020,"Comedy,Crime,Drama",nm2128335,director,6.3,132623.0,J Blakeson
68482,3363431,tt9894470,VFW,2019,"Action,Crime,Horror",nm3976308,director,6.1,8558.0,Joe Begos
68487,3363819,tt9898858,Coffee & Kareem,2020,"Action,Comedy,Crime",nm0236226,director,5.1,13801.0,Michael Dowse
68492,3364105,tt9900782,Kaithi,2019,"Action,Adventure,Crime",nm7992231,director,8.5,31732.0,Lokesh Kanagaraj


In [ ]:
## Fill all N/A with ''
imdb1 = imdb1.fillna('')

In [ ]:
## Group by Movie title, aggregate all the directors in a row (per movie) 
imdb2 = imdb1.groupby(['tconst', 'Year','IMDb rating']).agg({'Movie title' : 'first', 'Staff name' : ' '.join}).reset_index()
## Add new column movie title + staff name for join with RT
imdb2['dir_mov'] = imdb2['Movie title']+' '+imdb2['Staff name']
imdb2.head(20)

,tconst,Year,IMDb rating,Movie title,Staff name,dir_mov
0,tt0004972,1915,6.2,The Birth of a Nation,D.W. Griffith,The Birth of a Nation D.W. Griffith
1,tt0006864,1916,7.7,Intolerance,D.W. Griffith,Intolerance D.W. Griffith
2,tt0009968,1919,7.3,Broken Blossoms,D.W. Griffith,Broken Blossoms D.W. Griffith
3,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,Robert Wiene,The Cabinet of Dr. Caligari Robert Wiene
4,tt0011237,1920,7.2,The Golem,Carl Boese,The Golem Carl Boese
5,tt0012494,1921,7.6,Destiny,Fritz Lang,Destiny Fritz Lang
6,tt0013086,1922,7.8,"Dr. Mabuse, the Gambler",Fritz Lang,"Dr. Mabuse, the Gambler Fritz Lang"
7,tt0013427,1922,7.6,Nanook of the North,Robert J. Flaherty,Nanook of the North Robert J. Flaherty
8,tt0013442,1922,7.9,Nosferatu,F.W. Murnau,Nosferatu F.W. Murnau
9,tt0014341,1923,7.8,Our Hospitality,John G. Blystone,Our Hospitality John G. Blystone


In [ ]:
##Create new column dir_mov_set as a frozen set
imdb2['dir_mov_set'] = imdb2['dir_mov'].apply(lambda director : frozenset(director.split(' ')))
imdb2

,tconst,Year,IMDb rating,Movie title,Staff name,dir_mov,dir_mov_set
0,tt0004972,1915,6.2,The Birth of a Nation,D.W. Griffith,The Birth of a Nation D.W. Griffith,"(The, of, a, Nation, Griffith, D.W., Birth)"
1,tt0006864,1916,7.7,Intolerance,D.W. Griffith,Intolerance D.W. Griffith,"(Griffith, D.W., Intolerance)"
2,tt0009968,1919,7.3,Broken Blossoms,D.W. Griffith,Broken Blossoms D.W. Griffith,"(D.W., Broken, Blossoms, Griffith)"
3,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,Robert Wiene,The Cabinet of Dr. Caligari Robert Wiene,"(of, Robert, Dr., Cabinet, Wiene, The, Caligari)"
4,tt0011237,1920,7.2,The Golem,Carl Boese,The Golem Carl Boese,"(Boese, Golem, Carl, The)"
...,...,...,...,...,...,...,...
13001,tt9893250,2020,6.3,I Care a Lot,J Blakeson,I Care a Lot J Blakeson,"(I, Care, Lot, Blakeson, J, a)"
13002,tt9894470,2019,6.1,VFW,Joe Begos,VFW Joe Begos,"(VFW, Joe, Begos)"
13003,tt9898858,2020,5.1,Coffee & Kareem,Michael Dowse,Coffee & Kareem Michael Dowse,"(&, Coffee, Michael, Dowse, Kareem)"
13004,tt9900782,2019,8.5,Kaithi,Lokesh Kanagaraj,Kaithi Lokesh Kanagaraj,"(Kaithi, Kanagaraj, Lokesh)"


In [ ]:
rt

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,dir_mov
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,Chris ColumbusPercy Jackson & the Olympians: T...
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,Nicole HolofcenerPlease Give
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,2014-07-24,122.0,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,Blake Edwards10
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,2017-01-13,95.0,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,Sidney Lumet12 Angry Men (Twelve Angry Men)
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,2016-06-10,127.0,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,"Richard Fleischer20,000 Leagues Under The Sea"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"Drama, Musical & Performing Arts",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,2016-04-19,104.0,MCA Universal Home Video,Rotten,56.0,9.0,Upright,74.0,1195.0,Luis ValdezZoot Suit
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"Action & Adventure, Animation, Comedy","Byron Howard, Rich Moore, Jared Bush","Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,2016-06-07,108.0,Walt Disney Animation Studios,Certified-Fresh,98.0,291.0,Upright,92.0,101511.0,"Byron Howard, Rich Moore, Jared BushZootopia"
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"Action & Adventure, Art House & International,...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,2015-11-25,142.0,Fox,Fresh,80.0,10.0,Upright,86.0,7146.0,NaN
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"Classics, Drama","Cy Endfield, Cyril Endfield","Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,2017-01-08,135.0,Paramount Pictures,Fresh,96.0,23.0,Upright,91.0,30193.0,"Cy Endfield, Cyril EndfieldZulu"


In [ ]:
## Add new column movie title + director name for join with IMDB
rt['directors']=rt['directors'].str.replace(',','')
rt['dir_mov']=rt['movie_title']+' '+rt['directors']
rt

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,dir_mov
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,Percy Jackson & the Olympians: The Lightning T...
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,2012-09-04,90.0,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,Please Give Nicole Holofcener
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,2014-07-24,122.0,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,10 Blake Edwards
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,2017-01-13,95.0,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,12 Angry Men (Twelve Angry Men) Sidney Lumet
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,2016-06-10,127.0,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,"20,000 Leagues Under The Sea Richard Fleischer"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Mexican-American gangster Henry Reyna (Daniel ...,NaN,R,"Drama, Musical & Performing Arts",Luis Valdez,Luis Valdez,"Daniel Valdez, Edward James Olmos, Charles Aid...",1981-10-02,2016-04-19,104.0,MCA Universal Home Video,Rotten,56.0,9.0,Upright,74.0,1195.0,Zoot Suit Luis Valdez
17708,m/zootopia,Zootopia,From the largest elephant to the smallest shre...,The brilliantly well-rounded Zootopia offers a...,PG,"Action & Adventure, Animation, Comedy",Byron Howard Rich Moore Jared Bush,"Jared Bush, Phil Johnston","J.K. Simmons, Kristen Bell, Octavia Spencer, A...",2016-03-04,2016-06-07,108.0,Walt Disney Animation Studios,Certified-Fresh,98.0,291.0,Upright,92.0,101511.0,Zootopia Byron Howard Rich Moore Jared Bush
17709,m/zorba_the_greek,Zorba the Greek,Traveling to inspect an abandoned mine his fat...,NaN,NR,"Action & Adventure, Art House & International,...",NaN,NaN,"Anthony Quinn, Alan Bates, Irene Papas, Lila K...",1964-12-17,2015-11-25,142.0,Fox,Fresh,80.0,10.0,Upright,86.0,7146.0,NaN
17710,m/zulu,Zulu,"In 1879, the Zulu nation hands colonial Britis...",Zulu patiently establishes a cast of colorful ...,PG,"Classics, Drama",Cy Endfield Cyril Endfield,"Cy Endfield, John Prebble","Stanley Baker, Jack Hawkins, Ulla Jacobsson, J...",1964-06-17,2017-01-08,135.0,Paramount Pictures,Fresh,96.0,23.0,Upright,91.0,30193.0,Zulu Cy Endfield Cyril Endfield


In [ ]:
## Keep only necessary columns
rt_final= rt[['rotten_tomatoes_link', 'movie_title', 'directors', 'tomatometer_rating', 'dir_mov']]
rt_final

,rotten_tomatoes_link,movie_title,directors,tomatometer_rating,dir_mov
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Chris Columbus,49.0,Percy Jackson & the Olympians: The Lightning T...
1,m/0878835,Please Give,Nicole Holofcener,87.0,Please Give Nicole Holofcener
2,m/10,10,Blake Edwards,67.0,10 Blake Edwards
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Sidney Lumet,100.0,12 Angry Men (Twelve Angry Men) Sidney Lumet
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea",Richard Fleischer,89.0,"20,000 Leagues Under The Sea Richard Fleischer"
...,...,...,...,...,...
17707,m/zoot_suit,Zoot Suit,Luis Valdez,56.0,Zoot Suit Luis Valdez
17708,m/zootopia,Zootopia,Byron Howard Rich Moore Jared Bush,98.0,Zootopia Byron Howard Rich Moore Jared Bush
17709,m/zorba_the_greek,Zorba the Greek,NaN,80.0,NaN
17710,m/zulu,Zulu,Cy Endfield Cyril Endfield,96.0,Zulu Cy Endfield Cyril Endfield


In [ ]:
## Create new column dir_mov_set as a frozen set
rt['dir_mov_set'] = rt['dir_mov'].fillna('').apply(lambda director : frozenset(director.split(' ')))
#rt

In [ ]:
rt['Year'] = rt['original_release_date'].astype(str)
rt['Year'] = rt['Year'].apply(lambda x : x[0:4])


In [ ]:
#print(rt_final['dir_mov'].isna().sum())
#rt_final['dir_mov'].value_counts().head(10)

In [ ]:
#rt_final.isna().sum()
#rtfinal1 = rt_final.fillna('')
#rtfinal1.isna().sum()
#rtfinal1

In [ ]:
## Join IMDB and RT tables
df = pd.merge(imdb2, rt, how='left', on='dir_mov_set')
df

,tconst,Year_x,IMDb rating,Movie title,Staff name,dir_mov_x,dir_mov_set,rotten_tomatoes_link,movie_title,movie_info,...,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,dir_mov_y,Year_y
0,tt0004972,1915,6.2,The Birth of a Nation,D.W. Griffith,The Birth of a Nation D.W. Griffith,"(The, of, a, Nation, Griffith, D.W., Birth)",m/birth_of_a_nation,The Birth of a Nation,"Two families, abolitionist Northerners the Sto...",...,190.0,Gravitas,Fresh,93.0,44.0,Spilled,56.0,4804.0,The Birth of a Nation D.W. Griffith,1915
1,tt0006864,1916,7.7,Intolerance,D.W. Griffith,Intolerance D.W. Griffith,"(Griffith, D.W., Intolerance)",m/1010644-intolerance,Intolerance,D.W. Griffith's epic intercuts between four se...,...,178.0,Cohen Media Group,Fresh,97.0,37.0,Upright,77.0,4908.0,Intolerance D.W. Griffith,1916
2,tt0009968,1919,7.3,Broken Blossoms,D.W. Griffith,Broken Blossoms D.W. Griffith,"(D.W., Broken, Blossoms, Griffith)",m/1003120-broken_blossoms,Broken Blossoms,When Chinese immigrant Cheng Huan (Richard Bar...,...,87.0,Kino on Video,Fresh,95.0,22.0,Upright,71.0,3802.0,Broken Blossoms D.W. Griffith,1919
3,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,Robert Wiene,The Cabinet of Dr. Caligari Robert Wiene,"(of, Robert, Dr., Cabinet, Wiene, The, Caligari)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0011237,1920,7.2,The Golem,Carl Boese,The Golem Carl Boese,"(Boese, Golem, Carl, The)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13008,tt9893250,2020,6.3,I Care a Lot,J Blakeson,I Care a Lot J Blakeson,"(I, Care, Lot, Blakeson, J, a)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13009,tt9894470,2019,6.1,VFW,Joe Begos,VFW Joe Begos,"(VFW, Joe, Begos)",m/vfw,VFW,A typical night for veterans at a VFW turns in...,...,92.0,RLJE Films,Fresh,81.0,54.0,Upright,61.0,133.0,VFW Joe Begos,2020
13010,tt9898858,2020,5.1,Coffee & Kareem,Michael Dowse,Coffee & Kareem Michael Dowse,"(&, Coffee, Michael, Dowse, Kareem)",m/coffee_and_kareem,Coffee & Kareem,A 12-year-old's scheme to scare away his mom's...,...,88.0,Netflix,Rotten,20.0,69.0,Spilled,30.0,398.0,Coffee & Kareem Michael Dowse,nan
13011,tt9900782,2019,8.5,Kaithi,Lokesh Kanagaraj,Kaithi Lokesh Kanagaraj,"(Kaithi, Kanagaraj, Lokesh)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['rotten_tomatoes_link'].isna().sum()

5761

In [ ]:
## Create DF3 just with N/A on RT, but without any info from RT
df2 = df[df['rotten_tomatoes_link'].isna()]
df3 = df2[['tconst', 'Year_x','IMDb rating','Movie title', 'Staff name', 'dir_mov_x', 'dir_mov_set']]
df3['Year2'] = df3['Year_x'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5761 entries, 3 to 13012
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tconst       5761 non-null   object 
 1   Year_x       5761 non-null   int64  
 2   IMDb rating  5761 non-null   float64
 3   Movie title  5761 non-null   object 
 4   Staff name   5761 non-null   object 
 5   dir_mov_x    5761 non-null   object 
 6   dir_mov_set  5761 non-null   object 
 7   Year2        5761 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 405.1+ KB


In [ ]:
## New column with first 15 characters of movie title + Year -- RT table
rt['title_firstchar']=rt['movie_title'].apply(lambda x : x[0:15])
rt['firstchar_year']=rt['title_firstchar']+' '+rt['Year']


In [ ]:
## New column with first 15 characters of movie title + Year  -- DF3 table
df3['title_firstchar']=df3['Movie title'].apply(lambda x : x[0:15])
df3
df3['firstchar_year']=df3['title_firstchar']+' '+df3['Year2']
df3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,tconst,Year_x,IMDb rating,Movie title,Staff name,dir_mov_x,dir_mov_set,Year2,title_firstchar,firstchar_year
3,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,Robert Wiene,The Cabinet of Dr. Caligari Robert Wiene,"(of, Robert, Dr., Cabinet, Wiene, The, Caligari)",1920,The Cabinet of,The Cabinet of 1920
4,tt0011237,1920,7.2,The Golem,Carl Boese,The Golem Carl Boese,"(Boese, Golem, Carl, The)",1920,The Golem,The Golem 1920
5,tt0012494,1921,7.6,Destiny,Fritz Lang,Destiny Fritz Lang,"(Fritz, Destiny, Lang)",1921,Destiny,Destiny 1921
6,tt0013086,1922,7.8,"Dr. Mabuse, the Gambler",Fritz Lang,"Dr. Mabuse, the Gambler Fritz Lang","(the, Gambler, Dr., Lang, Mabuse,, Fritz)",1922,"Dr. Mabuse, the","Dr. Mabuse, the 1922"
8,tt0013442,1922,7.9,Nosferatu,F.W. Murnau,Nosferatu F.W. Murnau,"(F.W., Murnau, Nosferatu)",1922,Nosferatu,Nosferatu 1922
...,...,...,...,...,...,...,...,...,...,...
13006,tt9866072,2020,6.1,Holidate,John Whitesell,Holidate John Whitesell,"(John, Whitesell, Holidate)",2020,Holidate,Holidate 2020
13007,tt9877170,2020,6.5,Malang,Mohit Suri,Malang Mohit Suri,"(Suri, Malang, Mohit)",2020,Malang,Malang 2020
13008,tt9893250,2020,6.3,I Care a Lot,J Blakeson,I Care a Lot J Blakeson,"(I, Care, Lot, Blakeson, J, a)",2020,I Care a Lot,I Care a Lot 2020
13011,tt9900782,2019,8.5,Kaithi,Lokesh Kanagaraj,Kaithi Lokesh Kanagaraj,"(Kaithi, Kanagaraj, Lokesh)",2019,Kaithi,Kaithi 2019


In [ ]:
## Join df3 (N/A) and rt on the first 15 charac of the movie title and year
imdb_rt_nan = pd.merge(df3, rt, how='left', on='firstchar_year')
imdb_rt_nan.dropna(inplace=True)

imdb_rt_nan.drop(['Year2','title_firstchar_x','firstchar_year','dir_mov',	'dir_mov_set_y', 'title_firstchar_y','Year'], axis = 1, inplace = True) 
imdb_rt_nan

,tconst,Year_x,IMDb rating,Movie title,Staff name,dir_mov_x,dir_mov_set_x,rotten_tomatoes_link,movie_title,movie_info,...,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count
0,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,Robert Wiene,The Cabinet of Dr. Caligari Robert Wiene,"(of, Robert, Dr., Cabinet, Wiene, The, Caligari)",m/the_cabinet_of_dr_caligari,The Cabinet of Dr. Caligari (Das Cabinet des D...,"At a carnival in Germany, Francis (Friedrich F...",...,1920-02-26,2016-03-22,69.0,Rialto Pictures,Certified-Fresh,98.0,62.0,Upright,89.0,27748.0
6,tt0014429,1923,8.1,Safety Last!,Fred C. Newmeyer Sam Taylor,Safety Last! Fred C. Newmeyer Sam Taylor,"(Taylor, C., Last!, Safety, Fred, Sam, Newmeyer)",m/safety_last,Safety Last!,A boy (Harold Lloyd) moves to New York City to...,...,1923-04-01,2017-03-17,74.0,Criterion Collection,Fresh,97.0,35.0,Upright,93.0,3749.0
12,tt0015648,1925,8.0,Battleship Potemkin,Sergei Eisenstein,Battleship Potemkin Sergei Eisenstein,"(Sergei, Eisenstein, Battleship, Potemkin)",m/battleship_potemkin,Battleship Potemkin,"When they are fed rancid meat, the sailors on ...",...,1925-12-24,2016-04-08,67.0,Kino International,Certified-Fresh,100.0,48.0,Upright,86.0,18920.0
21,tt0018578,1927,7.6,Wings,William A. Wellman Harry d'Abbadie d'Arrast,Wings William A. Wellman Harry d'Abbadie d'Arrast,"(William, d'Arrast, Harry, d'Abbadie, Wellman,...",m/wings,Wings,"With World War I afoot, David Armstrong (Richa...",...,1927-08-12,2017-01-02,139.0,Unknown,Certified-Fresh,93.0,46.0,Upright,78.0,3530.0
26,tt0019254,1928,8.2,The Passion of Joan of Arc,Carl Theodor Dreyer,The Passion of Joan of Arc Carl Theodor Dreyer,"(of, Dreyer, Carl, Passion, Joan, Theodor, Arc...",m/passion_of_joan_of_arc,The Passion of Joan of Arc (La Passion de Jean...,"A classic of the silent age, this film tells t...",...,1928-04-21,2018-07-23,77.0,Criterion Collection,Certified-Fresh,98.0,55.0,Upright,93.0,10385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,tt6359956,2017,5.6,A Bad Moms Christmas,Jon Lucas Scott Moore,A Bad Moms Christmas Jon Lucas Scott Moore,"(Bad, Christmas, Jon, Moore, Scott, Moms, A, L...",m/a_bad_moms_christmas,A Bad Moms Christmas,"Under-appreciated and overburdened moms Amy, K...",...,2017-11-01,2018-01-23,104.0,STXfilms,Rotten,32.0,129.0,Spilled,44.0,13280.0
5301,tt7170950,2018,6.1,Like Father,Lauren Miller Rogen,Like Father Lauren Miller Rogen,"(Lauren, Like, Rogen, Miller, Father)",m/like_father_2018,Like Father,"Left at the altar, a young executive takes her...",...,2018-08-03,2018-08-03,103.0,Netflix,Rotten,46.0,41.0,Spilled,46.0,544.0
5315,tt7242142,2018,7.4,Blindspotting,Carlos López Estrada,Blindspotting Carlos López Estrada,"(Estrada, Blindspotting, Carlos, López)",m/blindspotting,Blindspotting,Collin must make it through his final three da...,...,2018-07-20,2018-11-06,95.0,Foley Walkers Studio,Certified-Fresh,94.0,171.0,Upright,87.0,2400.0
5446,tt7946422,2018,6.3,Prospect,Christopher Caldwell Zeek Earl,Prospect Christopher Caldwell Zeek Earl,"(Christopher, Zeek, Earl, Prospect, Caldwell)",m/prospect_2018,Prospect,A man and his teenage daughter encounter nonst...,...,2018-11-02,2019-03-08,98.0,DUST,Certified-Fresh,88.0,49.0,Upright,70.0,1092.0


In [ ]:
## Create df_not_nan by dropping N/A on df 
df_not_nan = df
df_not_nan.dropna(subset=['rotten_tomatoes_link'],inplace=True)
df_not_nan.drop(['dir_mov_y',	'Year_y'], axis = 1, inplace = True)
df_not_nan

,tconst,Year_x,IMDb rating,Movie title,Staff name,dir_mov_x,dir_mov_set,rotten_tomatoes_link,movie_title,movie_info,...,original_release_date,streaming_release_date,runtime,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count
0,tt0004972,1915,6.2,The Birth of a Nation,D.W. Griffith,The Birth of a Nation D.W. Griffith,"(The, of, a, Nation, Griffith, D.W., Birth)",m/birth_of_a_nation,The Birth of a Nation,"Two families, abolitionist Northerners the Sto...",...,1915-03-03,2016-07-08,190.0,Gravitas,Fresh,93.0,44.0,Spilled,56.0,4804.0
1,tt0006864,1916,7.7,Intolerance,D.W. Griffith,Intolerance D.W. Griffith,"(Griffith, D.W., Intolerance)",m/1010644-intolerance,Intolerance,D.W. Griffith's epic intercuts between four se...,...,1916-09-05,2016-08-11,178.0,Cohen Media Group,Fresh,97.0,37.0,Upright,77.0,4908.0
2,tt0009968,1919,7.3,Broken Blossoms,D.W. Griffith,Broken Blossoms D.W. Griffith,"(D.W., Broken, Blossoms, Griffith)",m/1003120-broken_blossoms,Broken Blossoms,When Chinese immigrant Cheng Huan (Richard Bar...,...,1919-05-13,2016-08-11,87.0,Kino on Video,Fresh,95.0,22.0,Upright,71.0,3802.0
7,tt0013427,1922,7.6,Nanook of the North,Robert J. Flaherty,Nanook of the North Robert J. Flaherty,"(the, of, Robert, Flaherty, Nanook, J., North)",m/nanook_of_the_north,Nanook of the North,This pioneering documentary film depicts the l...,...,1922-06-11,2018-04-03,79.0,NaN,Fresh,100.0,30.0,Upright,80.0,4871.0
14,tt0015400,1924,7.7,The Thief of Bagdad,Raoul Walsh,The Thief of Bagdad Raoul Walsh,"(of, Bagdad, Walsh, Thief, The, Raoul)",m/the_thief_of_bagdad,The Thief of Bagdad,When the Thief of Baghdad (Douglas Fairbanks) ...,...,1924-03-18,2016-02-18,155.0,United Artists,Fresh,96.0,27.0,Upright,82.0,2299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12993,tt9806192,2019,7.5,I Lost My Body,Jérémy Clapin,I Lost My Body Jérémy Clapin,"(I, Clapin, Lost, My, Body, Jérémy)",m/i_lost_my_body,I Lost My Body,"A severed hand escapes from a dissection lab, ...",...,2019-11-15,2019-11-29,81.0,Netflix,Certified-Fresh,96.0,81.0,Upright,84.0,561.0
12995,tt9806370,2020,4.9,The Owners,Julius Berg,The Owners Julius Berg,"(Julius, Owners, Berg, The)",m/the_owners_2020,The Owners,An elderly couple turn the tables on a group o...,...,2020-09-04,2020-09-04,92.0,RLJE Films,Fresh,64.0,39.0,Spilled,31.0,65.0
13005,tt9860728,2019,5.6,Falling Inn Love,Roger Kumble,Falling Inn Love Roger Kumble,"(Roger, Inn, Kumble, Love, Falling)",m/falling_inn_love,Falling Inn Love,Romance and remodeling collide when a corporat...,...,NaN,2019-08-29,98.0,NaN,Fresh,65.0,20.0,Spilled,32.0,235.0
13009,tt9894470,2019,6.1,VFW,Joe Begos,VFW Joe Begos,"(VFW, Joe, Begos)",m/vfw,VFW,A typical night for veterans at a VFW turns in...,...,2020-02-14,2020-02-14,92.0,RLJE Films,Fresh,81.0,54.0,Upright,61.0,133.0


In [ ]:
imdb_rt_nan['rotten_tomatoes_link'].isna().sum()

0

In [ ]:
## Concat between 2 tables with RT and IMDB ratings
final = pd.concat([df_not_nan, imdb_rt_nan])
imdb_rt_ratings = final[['tconst','Year_x','IMDb rating','Movie title', 'rotten_tomatoes_link', 'movie_title', 'tomatometer_status','tomatometer_rating']]
imdb_rt_ratings

,tconst,Year_x,IMDb rating,Movie title,rotten_tomatoes_link,movie_title,tomatometer_status,tomatometer_rating
0,tt0004972,1915,6.2,The Birth of a Nation,m/birth_of_a_nation,The Birth of a Nation,Fresh,93.0
1,tt0006864,1916,7.7,Intolerance,m/1010644-intolerance,Intolerance,Fresh,97.0
2,tt0009968,1919,7.3,Broken Blossoms,m/1003120-broken_blossoms,Broken Blossoms,Fresh,95.0
7,tt0013427,1922,7.6,Nanook of the North,m/nanook_of_the_north,Nanook of the North,Fresh,100.0
14,tt0015400,1924,7.7,The Thief of Bagdad,m/the_thief_of_bagdad,The Thief of Bagdad,Fresh,96.0
...,...,...,...,...,...,...,...,...
5143,tt6359956,2017,5.6,A Bad Moms Christmas,m/a_bad_moms_christmas,A Bad Moms Christmas,Rotten,32.0
5301,tt7170950,2018,6.1,Like Father,m/like_father_2018,Like Father,Rotten,46.0
5315,tt7242142,2018,7.4,Blindspotting,m/blindspotting,Blindspotting,Certified-Fresh,94.0
5446,tt7946422,2018,6.3,Prospect,m/prospect_2018,Prospect,Certified-Fresh,88.0


In [ ]:
#imdb_rt_ratings.to_csv('imdb_rt_ratings.csv',index = False)

In [ ]:
#imdb2.to_csv('imdb_5percent.csv',index = False)

**OUTPUT 2 - Top movies per director**

In [ ]:
directors = pd.merge(imdb1, imdb_rt_ratings, how='left', on='tconst')
directors

,Unnamed: 0,tconst,Movie title_x,Year,Genres,nconst,category,IMDb rating_x,Nb votes,Staff name,Year_x,IMDb rating_y,Movie title_y,rotten_tomatoes_link,movie_title,tomatometer_status,tomatometer_rating
0,4320,tt0004972,The Birth of a Nation,1915,"Drama,History,War",nm0000428,director,6.2,24922.0,D.W. Griffith,1915.0,6.2,The Birth of a Nation,m/birth_of_a_nation,The Birth of a Nation,Fresh,93.0
1,10770,tt0006864,Intolerance,1916,"Drama,History",nm0000428,director,7.7,15692.0,D.W. Griffith,1916.0,7.7,Intolerance,m/1010644-intolerance,Intolerance,Fresh,97.0
2,25195,tt0009968,Broken Blossoms,1919,"Drama,Romance",nm0000428,director,7.3,10437.0,D.W. Griffith,1919.0,7.3,Broken Blossoms,m/1003120-broken_blossoms,Broken Blossoms,Fresh,95.0
3,26838,tt0010323,The Cabinet of Dr. Caligari,1920,"Horror,Mystery,Thriller",nm0927468,director,8.0,64319.0,Robert Wiene,1920.0,8.0,The Cabinet of Dr. Caligari,m/the_cabinet_of_dr_caligari,The Cabinet of Dr. Caligari (Das Cabinet des D...,Certified-Fresh,98.0
4,31277,tt0011237,The Golem,1920,"Fantasy,Horror",nm0091380,director,7.2,7825.0,Carl Boese,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14227,3363277,tt9893250,I Care a Lot,2020,"Comedy,Crime,Drama",nm2128335,director,6.3,132623.0,J Blakeson,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14228,3363431,tt9894470,VFW,2019,"Action,Crime,Horror",nm3976308,director,6.1,8558.0,Joe Begos,2019.0,6.1,VFW,m/vfw,VFW,Fresh,81.0
14229,3363819,tt9898858,Coffee & Kareem,2020,"Action,Comedy,Crime",nm0236226,director,5.1,13801.0,Michael Dowse,2020.0,5.1,Coffee & Kareem,m/coffee_and_kareem,Coffee & Kareem,Rotten,20.0
14230,3364105,tt9900782,Kaithi,2019,"Action,Adventure,Crime",nm7992231,director,8.5,31732.0,Lokesh Kanagaraj,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## Clean dataset: keep only necessary columns and convert Year to integer
directors1 = directors[['tconst','Year','IMDb rating_x','Movie title_x', 'rotten_tomatoes_link', 'Staff name', 'movie_title', 'tomatometer_status','tomatometer_rating']]
directors1['Year'] = directors1['Year'].astype(str)
directors1['Year'] = directors1['Year'].apply(lambda x : x[0:4])
directors1['Year'] = directors1['Year'].apply(lambda x : int(x))
directors2 = directors1.rename(columns = {'IMDb rating_x':'IMDb rating', 'Movie title_x':'Movie title', 'tomatometer_rating':'Rotten Tomatoes rating', 'Staff name':'Director name'})
directors2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,tconst,Year,IMDb rating,Movie title,rotten_tomatoes_link,Director name,movie_title,tomatometer_status,Rotten Tomatoes rating
0,tt0004972,1915,6.2,The Birth of a Nation,m/birth_of_a_nation,D.W. Griffith,The Birth of a Nation,Fresh,93.0
1,tt0006864,1916,7.7,Intolerance,m/1010644-intolerance,D.W. Griffith,Intolerance,Fresh,97.0
2,tt0009968,1919,7.3,Broken Blossoms,m/1003120-broken_blossoms,D.W. Griffith,Broken Blossoms,Fresh,95.0
3,tt0010323,1920,8.0,The Cabinet of Dr. Caligari,m/the_cabinet_of_dr_caligari,Robert Wiene,The Cabinet of Dr. Caligari (Das Cabinet des D...,Certified-Fresh,98.0
4,tt0011237,1920,7.2,The Golem,NaN,Carl Boese,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14227,tt9893250,2020,6.3,I Care a Lot,NaN,J Blakeson,NaN,NaN,NaN
14228,tt9894470,2019,6.1,VFW,m/vfw,Joe Begos,VFW,Fresh,81.0
14229,tt9898858,2020,5.1,Coffee & Kareem,m/coffee_and_kareem,Michael Dowse,Coffee & Kareem,Rotten,20.0
14230,tt9900782,2019,8.5,Kaithi,NaN,Lokesh Kanagaraj,NaN,NaN,NaN


In [ ]:
#directors2.to_csv('directors_rt_imdb1.csv',index = False)